# Exercice 4: connexion à l'API Mistral
L’objectif de cet exercice est de développer un mini-assistant conversationnel en Python qui utilise l’API Mistral pour répondre à des questions ou effectuer des tâches simples (résumé de texte, traduction, génération de code, etc.).

Avant de commencer à regarder et utiliser le code ci-dessous, il faut créer une clé api sur le site de Mistral. Sur [console.mistral.ai](https://console.mistral.ai), créer une clé API pour lechat. Il faudra créer un compte et souscrire à un plan expérimental (gratuit) pour l’obtenir. Copier cette clé dans la variable API_KEY.

In [ ]:
%pip install mistralai
%pip install requests

In [ ]:
import os
import requests
from textwrap import dedent

# Remplace ces valeurs par celles de ton compte Mistral
API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "entre ta clé Mistral ici"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}

data = {
    "model": "mistral-tiny",  # Remplace par le modèle que tu veux utiliser
    "messages": [{"role": "user", "content": "Bonjour, comment ça va ?"}],
}

try:
    response = requests.post(API_URL, headers=headers, json=data)
    response.raise_for_status()  # Lève une erreur si la requête échoue
    print(response.json()["choices"][0]["message"]["content"])
except requests.exceptions.RequestException as e:
    print(f"Erreur lors de la requête : {e}")


Et voila, c'est tout, tu parles avec le chat! Maintenant, tansforme le code ci-dessous pour qu'il soit simple à utiliser. Créer une classe contenant une méthode pour initialiser la connexion à l'API et une pour envoyer un message au LLM et et récupérer son retour.

In [ ]:
def mistral_chat(
    messages,
    model="mistral-small-latest",  # tu peux mettre "mistral-tiny" si tu veux moins cher/plus rapide
    temperature=0.3,
    max_tokens=512,
    timeout=60,
):
    """
    messages: liste de dicts [{"role": "system|user|assistant", "content": "..."}]
    Retour: texte (str) de la réponse assistant.
    """
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    try:
        r = requests.post(API_URL, headers=HEADERS, json=payload, timeout=timeout)
        r.raise_for_status()
        data = r.json()

        # Format standard: choices[0].message.content
        return data["choices"][0]["message"]["content"]

    except requests.exceptions.HTTPError as e:
        # Tente d'afficher le contenu d'erreur si possible
        try:
            err = r.json()
        except Exception:
            err = r.text if "r" in locals() else ""
        raise RuntimeError(f"HTTPError: {e}\nServer response: {err}")

    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Request failed: {e}")


L'objectif du reste de ce TP est maintenant de créer l'assistant en ajoutant des méthodes à cette classe.

Il faut pouvoir:

1. Résumer un texte long
2. Traduire un paragraphe en français vers l'anglais (ou d'autres langues en paramètres)
3. Générer du code Python pour une tâche donnée

Toutes les fonctions doivent être sur différents exemples. D'autres tâches aux choix peuvent être ajoutées à l'assistant sur le même principe.



In [ ]:
def summarize_text(text, model="mistral-small-latest", max_words=120):
    """
    Résume un texte long en français.
    max_words: contrainte de longueur (approx).
    """
    system = "Tu es un assistant académique. Tu résumes de manière fidèle et concise, sans inventer."
    user = dedent(f"""
    Résume le texte suivant en ~{max_words} mots.
    - Conserve les points clés.
    - Pas de détails inutiles.
    - Pas d'invention.

    TEXTE:
    {text}
    """)

    return mistral_chat(
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        model=model,
        temperature=0.2,
        max_tokens=300,
    )

In [ ]:
long_text = dedent("""
L’apprentissage profond a transformé le traitement automatique des langues en permettant de modéliser des dépendances complexes.
Les modèles récurrents (RNN, LSTM, GRU) ont d’abord été largement utilisés pour traiter des séquences.
Ensuite, l’attention et les Transformers ont amélioré la parallélisation et la capture de dépendances longues.
Cependant, ces modèles nécessitent beaucoup de données et de calcul.
En pratique, on doit aussi considérer la robustesse, les biais, et l’évaluation sur des données hors distribution.
Enfin, le choix du modèle dépend du compromis entre performance, coût et contraintes d’implémentation.
""").strip()

print(summarize_text(long_text, max_words=90))

In [ ]:
def translate_text(
    text, source_lang="français", target_lang="anglais", model="mistral-small-latest"
):
    """
    Traduit un texte d'une langue vers une autre.
    """
    system = "Tu es un traducteur professionnel. Tu fournis une traduction naturelle, fidèle, et fluide."
    user = dedent(f"""
    Traduis le texte suivant du {source_lang} vers le {target_lang}.
    - Reste fidèle au sens.
    - Adapte le style pour que ce soit naturel dans la langue cible.
    - Ne rajoute pas d'explications, uniquement la traduction.

    TEXTE:
    {text}
    """)

    return mistral_chat(
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        model=model,
        temperature=0.2,
        max_tokens=400,
    )

In [ ]:
paragraph_fr = dedent("""
Dans cette expérience, nous évaluons un modèle séquentiel pour prédire le prochain caractère.
Nous comparons plusieurs architectures (LSTM, GRU) et analysons la perte de validation afin d’éviter un surapprentissage.
""").strip()

print(translate_text(paragraph_fr, source_lang="français", target_lang="anglais"))

In [ ]:
def generate_python_code(task_description, model="mistral-small-latest"):
    """
    Génère du code Python (avec explications minimales) pour une tâche donnée.
    Retourne UNIQUEMENT du code (pas de texte autour).
    """
    system = dedent("""
    Tu es un assistant Python. Tu réponds uniquement avec du code Python exécutable.
    - Pas de markdown, pas d'explication.
    - Code clair, robuste, avec fonctions et docstrings si utile.
    - Évite les dépendances non-standard sauf si demandé.
    """).strip()

    user = dedent(f"""
    Écris du code Python pour la tâche suivante:

    {task_description}

    Rappels:
    - Retourne uniquement du code Python.
    - Le code doit être complet et exécutable.
    """).strip()

    return mistral_chat(
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        model=model,
        temperature=0.1,
        max_tokens=700,
    )

In [ ]:
task = """
Charger un fichier CSV (chemin fourni), afficher:
- le nombre de lignes/colonnes
- les 5 premières lignes
- le nombre de valeurs manquantes par colonne
Puis sauvegarder un nouveau CSV nettoyé où les lignes contenant au moins une valeur manquante sont supprimées.
Le chemin du CSV d'entrée et de sortie doivent être des variables en début de script.
"""

code = generate_python_code(task)
print(code)